In [1]:
%run functions

c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
loader = DataLoader()
d = loader.GetData('ASAP')

In [3]:
embedding = [[] for i in range(8)]
for i in range(1,9):
    e = torch.load('features/asap-{}-cls.pt'.format(i))
    embedding[i-1]=[np.array(x).reshape(1024) for x in e]

In [4]:
large = dict()
with open('features/ASAP-l2sca.txt','r') as f:
    line = f.readlines()[1:]
for i in line:
    xi = i.split(',')
    ind = int(xi[0].split('.')[0])
    large[ind] = [float(s) for s in xi[-14:]]

In [1]:
fine =  dict()
with open('features/asap-fine.csv','r') as f:
    line = f.readlines()
for i in line:
    xi = i.split(',')
    ind = int(xi[0])
    fine[ind] = [float(s) for s in xi[1:]]

In [3]:
fine_taassc =  dict()
with open('features/asap-tasscs.csv','r') as f:
    line = f.readlines()[1:]
for i in line:
    xi = i.split(',')
    ind = int(xi[0].split('.')[0])
    fine_taassc[ind] = [float(s) for s in xi[1:]]

In [4]:
len(fine_taassc[0])

149

In [7]:
xsim = [[] for i in range(8)]
for t in range(1,9):
    f = np.load('features/asap-{}-sim.npz'.format(t))
    sim_list = [f[a] for a in f.files]
    sim_raw_list = []
    for n in f.files:
        values = []
        m = f[n]
        for row in range(len(m)):
            for col in range(row+1,len(m)):
                values.append(m[row,col])
        sim_raw_list.append(np.array(values))
    sim_mean = []
    sim_max = []
    sim_min = []
    for matrix in sim_raw_list:
        if len(matrix) > 1:
            sim_mean.append(np.average(matrix))
            sim_max.append(np.max(matrix))
            sim_min.append(np.min(matrix))
        else:
            sim_mean.append(0)
            sim_max.append(0)
            sim_min.append(0)
    cluster_num = []
    cluster_avg = []
    for k in range(len(sim_list)):
        sim = sim_list[k]
        if len(sim) > 1:
            sim_thres = sim.copy()
            for i in range(len(sim_thres)):
                for j in range(len(sim_thres)):
                    if sim_thres[i,j] <= np.percentile(np.array(sim_raw_list[k]),50):
                        sim_thres[i,j] = 0
            res = mcl.run_mcl(sim_thres,inflation=2)
            clusters = mcl.get_clusters(res)
            cluster_num.append(len(clusters))
            cluster_avg.append(len(sim)/len(clusters))
        else:
            cluster_num.append(0)
            cluster_avg.append(0)
    print(len(sim_max))
    for j in range(len(sim_max)):
        a = [sim_max[j],sim_mean[j],sim_min[j],cluster_avg[j],cluster_num[j]]
        xsim[t-1].append(a)

1783
1800
1726
1772
1805
1800
1569
723


In [8]:
unique = []
for i in set(d[d['essay_set']== 8]['domain1_score']):
    for ind in d.index[d['essay_set']== 8]:
        if d['domain1_score'][ind] == i:
            unique.append(ind)
            break

In [16]:
def classification(x,i):
    sp = int(len(x)*0.75)
    le = LabelEncoder()
    le.fit(d.loc[d['essay_set'] == i,'domain1_score'])
    y = le.transform(d.loc[d['essay_set'] ==i,'domain1_score'])
    TrainY,TestY = y[:sp],y[sp:]
    TrainX,TestX = np.array(x[:sp]),np.array(x[sp:])
    if i == 8:
        TrainX = np.array(x[:sp] + [x[ind-12255] for ind in unique])
        TrainY = np.concatenate((y[:sp], [y[ind-12255] for ind in unique]),0)
    model = xgb.XGBClassifier(tree_method="hist",objective='multi:softmax',num_class=len(set(d.loc[d['essay_set'] == i,'domain1_score'])),learning_rate=0.05)
    model.fit(TrainX,TrainY)
    PredY = model.predict(TestX)
    PredTrueY = model.predict(TrainX)
    return (balanced_accuracy_score(TestY,PredY),cohen_kappa_score(TestY,PredY,weights='quadratic'), balanced_accuracy_score(TrainY,PredTrueY),cohen_kappa_score(TrainY,PredTrueY,weights='quadratic'))

In [17]:
testQWK = {i:[] for i in range(1,9)}
testAcc = {i:[] for i in range(1,9)}
trainQWK = {i:[] for i in range(1,9)}
trainAcc = {i:[] for i in range(1,9)}
names = ['sim','large','fine','taa','embedding','x_sim_large','x_sim_fine','x_sim_taa','x_sim_fine_em']
for i in range(8):
    x_em = embedding[i]
    x_sim = xsim[i]
    x_large = [large[ind] for ind in d.index[d['essay_set'] == i+1]]
    x_fine = [fine[ind] for ind in d.index[d['essay_set'] == i+1]]
    x_taa = [fine_taassc[ind] for ind in d.index[d['essay_set'] == i+1]]
    x_sim_large = [np.concatenate((x_large[i],x_sim[i]),0) for i in range(len(x_sim))]  
    x_sim_fine = [np.concatenate((x_fine[i],x_sim[i]),0) for i in range(len(x_sim))]
    x_sim_taa = [np.concatenate((x_taa[i],x_sim[i]),0) for i in range(len(x_sim))]  
    x_sim_fine_em = [np.concatenate((x_em[i],x_sim_fine[i]),0) for i in range(len(x_sim))] 
    for x in [x_sim,x_large,x_fine,x_taa,x_em,x_sim_large,x_sim_fine,x_sim_taa,x_sim_fine_em]:
        res = classification(x,i+1)
        testQWK[i+1].append(res[1])
        testAcc[i+1].append(res[0])
        trainQWK[i+1].append(res[3])
        trainAcc[i+1].append(res[2])
    print(i)

0
1
2
3
4
5


c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


6


c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: Use

7


c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [18]:
pd.DataFrame({i:trainQWK[i] for i in range(1,9)},index = names)

,1,2,3,4,5,6,7,8
sim,0.894512,0.851328,0.832506,0.790114,0.890971,0.791005,0.906883,0.943468
large,0.865686,0.891286,0.888988,0.893958,0.884259,0.852781,0.977948,0.989035
fine,0.997488,0.987301,0.975929,0.982338,0.994984,0.990352,1.000000,0.999897
taa,0.999843,1.000000,0.992960,0.998686,0.998458,0.998802,1.000000,1.000000
embedding,1.000000,0.998739,1.000000,1.000000,0.999615,0.995602,1.000000,1.000000
x_sim_large,0.978963,0.958267,0.939663,0.947760,0.975577,0.948606,0.997865,0.999354
x_sim_fine,0.995595,0.990475,0.976498,0.984066,0.997688,0.986305,1.000000,0.999897
x_sim_taa,1.000000,0.998739,0.993544,0.999125,0.997687,1.000000,1.000000,1.000000
x_sim_fine_em,1.000000,0.999369,1.000000,1.000000,0.999615,0.996400,1.000000,1.000000


In [36]:
df = pd.DataFrame({i:testQWK[i] for i in range(1,9)},index = names)
df

,1,2,3,4,5,6,7,8
sim,0.611144,0.586755,0.587856,0.611569,0.723877,0.558954,0.607472,0.417137
large,0.173021,0.318007,0.324457,0.498750,0.495038,0.367864,0.343377,0.360623
fine,0.784733,0.703774,0.683527,0.727281,0.783380,0.724397,0.759206,0.629691
taa,0.774426,0.655524,0.694021,0.719455,0.765601,0.676041,0.682987,0.605557
embedding,0.760795,0.674535,0.683611,0.719021,0.778476,0.663792,0.694471,0.530882
x_sim_large,0.632325,0.618675,0.647502,0.676931,0.710898,0.608389,0.616477,0.389317
x_sim_fine,0.781015,0.709522,0.694122,0.728099,0.778936,0.721292,0.739123,0.654252
x_sim_taa,0.771481,0.645274,0.689358,0.722126,0.779800,0.679274,0.676724,0.570447
x_sim_fine_em,0.781594,0.688335,0.677114,0.727067,0.786424,0.682226,0.724118,0.623082


In [41]:
for i in names:
    print('{:.3f}'.format(df[8][i]))

0.417
0.361
0.630
0.606
0.531
0.389
0.654
0.570
0.623


In [20]:
pd.DataFrame({i:trainAcc[i] for i in range(1,9)},index = names)

,1,2,3,4,5,6,7,8
sim,0.724525,0.711252,0.689609,0.686099,0.736755,0.647420,0.687351,0.862297
large,0.842701,0.814084,0.863187,0.876016,0.890164,0.817607,0.888793,0.932881
fine,0.996907,0.958773,0.985764,0.975841,0.994184,0.983916,1.000000,0.970588
taa,0.999623,1.000000,0.989245,0.997786,0.997763,0.997923,1.000000,1.000000
embedding,1.000000,0.999421,1.000000,1.000000,0.999145,0.992098,1.000000,1.000000
x_sim_large,0.975459,0.919355,0.899710,0.924740,0.963905,0.928407,0.954317,0.970588
x_sim_fine,0.995067,0.960209,0.986004,0.977403,0.996788,0.976725,1.000000,0.970588
x_sim_taa,1.000000,0.999427,0.989755,0.998912,0.998251,1.000000,1.000000,1.000000
x_sim_fine_em,1.000000,0.999711,1.000000,1.000000,0.999145,0.993103,1.000000,1.000000


In [21]:
pd.DataFrame({i:testAcc[i] for i in range(1,9)},index = names)

,1,2,3,4,5,6,7,8
sim,0.299575,0.402646,0.454200,0.488066,0.502556,0.332182,0.103097,0.067917
large,0.140712,0.236621,0.333595,0.408877,0.346919,0.272760,0.088343,0.050920
fine,0.346887,0.479253,0.512652,0.587570,0.560664,0.507748,0.161678,0.077728
taa,0.385394,0.392645,0.535200,0.588550,0.498757,0.433427,0.125171,0.128131
embedding,0.375473,0.398838,0.521584,0.576675,0.526890,0.463017,0.152272,0.126675
x_sim_large,0.243190,0.357896,0.493277,0.554391,0.465966,0.383812,0.129904,0.081872
x_sim_fine,0.342963,0.497279,0.516750,0.595113,0.552312,0.500070,0.148605,0.082666
x_sim_taa,0.409565,0.384705,0.531930,0.593233,0.528058,0.441949,0.123905,0.116689
x_sim_fine_em,0.363234,0.437850,0.518247,0.588142,0.529980,0.461168,0.169603,0.120503
